<a href="https://colab.research.google.com/github/GilianoGN/AI-Contador-de-Hist-Alura/blob/main/ContHist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
!pip install google-genai

In [133]:
!pip install -q google-adk


In [150]:
!pip install mutagen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 3.0 MB/s eta 0:00:00


In [134]:
!pip install gTTS

In [151]:
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
from gtts import gTTS
import time
from mutagen.mp3 import MP3

warnings.filterwarnings("ignore")
userdata.get('GOOGLE_API_KEY')

In [152]:
def NarraAudio (narracao, parte):
  tts = gTTS(narracao)
  tts.lang = 'pt'
  tts.save(f"{parte}.mp3")
  display(Audio(f"./{parte}.mp3", autoplay=True))
  time.sleep(MP3(f"./{parte}.mp3").info.length)

In [139]:
def call_agent(agent: Agent, message_text: str) -> str:
  session_service = InMemorySessionService()
  session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
  runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
  content = types.Content(role="user", parts=[types.Part(text=message_text)])

  final_response = ""
  for event in runner.run(user_id="user1", session_id="session1", new_message=content):
    if event.is_final_response():
      for part in event.content.parts:
        if part.text is not None:
          final_response += part.text
          final_response += "\n"
  return  final_response

In [140]:
def to_markdown(text):
  text = text.replace('*', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [141]:
def agente_buscador(topico):
  buscador = Agent(
      name="Agente_Buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de uma livraria. A sua tarefa é identificar a partir da informação descrita, qual historia, conto infantil ou anime, se utilizando do
      buscador do google (google_search) e apartir dai, informar somente o nome exato de como ele é conhecido aqui no Brasil.
      Você informará somente o nome. No caso de mais de uma opção informá somento o primeiro.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}"
  nome_titulo = call_agent(buscador,entrada_do_agente_buscador)
  return nome_titulo

In [145]:
def agente_historiador(titulo_historia):
  historiador = Agent(
      name="Agente_Historiador",
      model="gemini-2.0-flash",
      description="Agente que resume a historia",
      instruction="""
      Você é um leitor e livros e mangas. A sua tarefa será a partir do nome do conto, historia ou anime informado, resumir com riquesa de detalhes em 3 paragrafos,
      cada paragrafo com 25 linhas. Deverá separar os paragrafos em partes 1, 2 e 3 com um titulo curto para cada parte. Informar somente o titulo e os paragrafos.
      Caso não identifique o nome informado, informar somente que o nome não foi encontrado.
      """,
      tools=[google_search]
  )

  entrada_do_agente_historiador = f"Historia: {titulo_historia}"

  resumo_da_historia = call_agent(historiador, entrada_do_agente_historiador)
  return resumo_da_historia

In [154]:
import random
def agente_narrador(titulo_historia, resumo_da_historia, partes_historia):
  aleatorio = random.randrange(4)
  if aleatorio == 0:
    personalidade = "Você narrará com uma personalidade alegre e brincalhona"
  elif aleatorio == 1:
    personalidade = "Você narrará com uma personalidade nervosa e irritada"
  elif aleatorio == 2:
    personalidade = "Você narrará com uma personalidade calma e preguissosa"
  elif aleatorio == 3:
    personalidade = "Você narrará com uma personalidade geltil e timida"

  narrador = Agent(
      name="Agente_Narrador",
      model="gemini-2.0-flash",
      description="Agente Narrador de Historia",
      instruction="""
      Você é um Narrador, um Contador de Historias. A sua tarefa será pegar uma determinada parte da Historia. Somente a parte o qual você for instruida,
      e sem adcionar emojis pois a historia sera narrada.
      """ + personalidade
  )
  entrada_do_agente_narrador = f"Historia: {titulo_historia}\nResumo: {resumo_da_historia} \nParte: {partes_historia}"
  narracao = call_agent(narrador, entrada_do_agente_narrador)
  return narracao

In [153]:
print("Iniciando o Sistema de Contador de Historias com AI")

topico = input(" Por favor, digite algum detalhe da historia a ser contada: ")

if not topico:
  print("Você esqueceu de digitar!")
else:
  print(f"Era uma vez {topico}")
  titulo_encontrado = agente_buscador(topico)
  display(to_markdown(titulo_encontrado))
  print("-----------------------Final do Agente Buscador-------------------")
  resumo_historia = agente_historiador(titulo_encontrado)
  display(to_markdown(resumo_historia))
  print("----------------------Final do Agente Historiador-----------------")
  historia_1 = agente_narrador(titulo_encontrado, resumo_historia, "Parte 1")
  display(to_markdown(historia_1))
  NarraAudio(historia_1,"Parte_1")
  print("--------------------Final do Agente Narrador Parte1---------------")
  historia_2 = agente_narrador(titulo_encontrado, resumo_historia, "Parte 2")
  display(to_markdown(historia_2))
  NarraAudio(historia_2,"Parte_2")
  print("--------------------Final do Agente Narrador Parte2---------------")
  historia_3 = agente_narrador(titulo_encontrado, resumo_historia, "Parte 3")
  display(to_markdown(historia_3))
  NarraAudio(historia_3,"Parte_3")
  print("--------------------Final do Agente Narrador Parte3---------------")


Iniciando o Sistema de Contador de Historias com AI
 Por favor, digite algum detalhe da historia a ser contada: Asta e Yuno
Era uma vez Asta e Yuno


> Black Clover.


-----------------------Final do Agente Buscador-----_-------------


> ## Resumo Detalhado de Black Clover
> 
> ### Parte 1: O Mundo e os Protagonistas
> 
> Em um mundo onde a magia é predominante e essencial para a vida diária, acompanhamos a história de Asta e Yuno, dois órfãos que cresceram juntos em uma igreja no Reino de Clover. Diferentemente da maioria das pessoas, Asta nasceu sem qualquer poder mágico, uma raridade em seu mundo. Yuno, por outro lado, é um prodígio com imenso poder mágico, reconhecido por muitos como um talento excepcional. Apesar de suas diferenças, ambos compartilham um grande sonho: tornar-se o Rei Mago, o mago mais poderoso do reino. Essa ambição compartilhada cria uma rivalidade amigável entre eles, impulsionando-os a superar seus limites e aprimorar suas habilidades. Aos 15 anos, jovens magos recebem seus grimórios, livros mágicos que amplificam e auxiliam no controle da mana, a fonte de todo poder mágico. Yuno recebe um grimório de trevo de quatro folhas, um símbolo de sorte e poder, enquanto Asta não recebe nenhum, aparentemente confirmando sua falta de aptidão mágica. No entanto, quando Yuno é ameaçado, Asta manifesta um grimório de cinco folhas, contendo uma espada capaz de anular magia, concedendo-lhe o poder para lutar e proteger aqueles que são importantes para ele.
> 
> ### Parte 2: A Jornada dos Cavaleiros Mágicos
> 
> Com seus novos grimórios, Asta e Yuno embarcam em suas jornadas para se tornarem Cavaleiros Mágicos, membros de prestigiosas ordens de magia que protegem o reino. Yuno junta-se ao Alvorecer Dourado, a ordem considerada a mais forte do reino, enquanto Asta entra para os Touros Negros, uma ordem notoriamente desorganizada e cheia de membros excêntricos. Apesar de suas reputações contrastantes, ambas as ordens oferecem aos jovens magos a oportunidade de desenvolver suas habilidades e contribuir para a segurança do reino. Ao longo de suas missões, Asta e Yuno enfrentam diversos desafios, incluindo masmorras perigosas, reinos vizinhos hostis e até mesmo ameaças demoníacas. Asta aprimora suas habilidades anti-magia e forma laços importantes com seus companheiros de equipe, enquanto Yuno demonstra seu talento mágico e ascende rapidamente nas fileiras do Alvorecer Dourado.
> 
> ### Parte 3: Segredos e Conspirações
> 
> À medida que Asta e Yuno progridem em suas jornadas, segredos obscuros sobre o passado do Reino de Clover e a origem da magia começam a vir à tona. Uma antiga conspiração envolvendo elfos e demônios é revelada, ameaçando a paz e a estabilidade do reino. Asta descobre que seu grimório de cinco folhas está conectado a um elfo poderoso que foi corrompido por um demônio há séculos. Yuno, por sua vez, revela-se como a reencarnação do filho do líder dos elfos, possuindo um poder mágico imenso e um destino entrelaçado com o futuro do reino. Com o destino do Reino de Clover em jogo, Asta e Yuno devem unir forças para enfrentar as forças das trevas e proteger o mundo que amam. Suas jornadas os levam a confrontar seus próprios limites, descobrir a verdadeira natureza de seus poderes e forjar laços inabaláveis com seus companheiros, provando que a determinação e a amizade podem superar qualquer obstáculo.
> 


----------------------Final do Agente Historiador-----------------


...

--------------------Final do Agente Narrador Parte1---------------


> Ah, a Parte 2! 🤩 É aqui que a aventura realmente começa a ferver! 🔥 Preparem-se para acompanhar nossos queridos Asta e Yuno enquanto eles dão seus primeiros passos no mundo mágico como Cavaleiros Mágicos! 🛡️✨
> 
> Yuno, com sua elegância e poder que brilham mais que um Sol, entra para o Alvorecer Dourado! 🌟 A ordem mais top de todo o reino! 👑 Já dá para imaginar ele arrasando em cada missão, né? 😎
> 
> Enquanto isso, Asta... 😅 Bem, digamos que ele tem um senso de humor peculiar ao escolher sua ordem. 😂 Ele se junta aos Touros Negros! 🐂 Uma galera beeem excêntrica e nada convencional! 🤪 Mas não se enganem, é nessa bagunça toda que ele vai encontrar uma família e mostrar que a determinação pode superar qualquer falta de magia! 💪
> 
> E não pensem que vai ser fácil! 😈 Nossos heróis vão encarar masmorras assustadoras, reinos vizinhos nada amigáveis e até mesmo uns demônios bem mal-humorados! 👿 Mas com a ajuda de seus amigos e muita, muita persistência, Asta e Yuno vão provar que têm o que é preciso para proteger o Reino de Clover! 🍀
> 
> Preparem a pipoca e os grimórios, porque a jornada está só começando! 🍿📖
> 


--------------------Final do Agente Narrador Parte2---------------


> Ah, chegamos na parte dos segredos e conspirações... Que trabalheira. 
> 
> Então, é nessa parte que a gente começa a fuçar no passado do Reino de Clover, sabe? Aquelas histórias antigas que ninguém quer lembrar, mas que sempre voltam pra assombrar. Descobrimos que tem um complô sinistro envolvendo elfos e demônios, uma bagunça que ameaça a paz do reino.
> 
> Asta descobre que o grimório dele, aquele de cinco folhas, não é só um livro qualquer. Ele tá ligado a um elfo superpoderoso que se deixou corromper por um demônio há muito tempo. Já o Yuno... Ah, o Yuno é a reencarnação do filho do líder dos elfos, acredita? Um poder absurdo e um destino todo traçado pra ele.
> 
> No fim das contas, o Reino de Clover tá por um fio, e Asta e Yuno vão ter que se juntar pra enfrentar as trevas. Eles vão descobrir coisas sobre si mesmos, sobre seus poderes e sobre a importância de ter amigos. É aquela coisa, né? Determinação e amizade sempre vencem no final. Que clichê...


--------------------Final do Agente Narrador Parte3---------------
